In [19]:
# optional - grab the environment variables from the current compute instance 
import os
import json
with open('/config.json', 'r') as f:
  env = json.loads(f.read())
os.environ['AZURE_SUBSCRIPTION_ID'] = env['subscription_id']
os.environ['AZURE_RESOURCE_GROUP'] = env['resource_group']
os.environ['AML_WORKSPACE'] = env['workspace_name']

In [ ]:
# # or alternativlely use dotenv to load the environment variables into the session from a custom .env file
# # 
# # (pip install python-dotenv)
# '''
# # Example .env file
# AZURE_SUBSCRIPTION_ID='subid'
# AZURE_RESOURCE_GROUP='rgname'
# AML_WORKSPACE='wsname'
# '''
# from dotenv import load_dotenv
# print('Loading environment variables from .env... Loaded:')
# load_dotenv('./.env')

In [20]:
import sys
print("Using Env: ", sys.executable)
print("Python verision: ", sys.version)

import os
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, DeviceCodeCredential
from azure.ai.ml import MLClient



# Connect using Default Credential - dependent on already being logged in via Azure CLI in the current environment
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    print("using fallback")
    # credential = InteractiveBrowserCredential()  
         ## does not work when working from Azure Databricks as we cannot launch a browser
    credential = DeviceCodeCredential(tenant_id=os.environ['AZURE_TENANT_ID'])  
         ## this may work if conditional access controlling login device types is not set by your tenant admin
    
# set up the mlclient
ml_client = MLClient(
    credential=credential,
    subscription_id=os.environ.get('AZURE_SUBSCRIPTION_ID'),  
    resource_group_name=os.environ.get('AZURE_RESOURCE_GROUP'),
    workspace_name=os.environ.get('AML_WORKSPACE'),
)

# verify access to the Azure Machine Learning Workspace by listing available compute
print("Available Compute: \n==================")
for c in ml_client.compute.list():
    print(c.name)

Using Env:  /anaconda/envs/azureml_py310_sdkv2/bin/python
Python verision:  3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Available Compute: 
cpu-cluster
DefLabelNC4asT4v3
cpu-cluster-ds12
cpu-cluster-lp
ds12-git
gpu-cluster-new
ds2-git


In [6]:
print("\nAvailable environments (examples):")
for e in ml_client.environments.list():
    # let's filter to keep it simple
    if 'sklearn' in e.name:   
    # or alternatively exclude curated envs
    # if e.creation_context.created_by_type == 'User' and e.creation_context.created_by != "Microsoft":    
        print(e.name, e.latest_version)


Available environments (examples):
AzureML-sklearn-1.0-ubuntu20.04-py38-cpu 36
AzureML-sklearn-0.24-ubuntu18.04-py37-cpu 49


In [23]:
from azure.ai.ml import load_environment
my_env = load_environment('../definitions/environments/base-image-plus-conda-file/definition.yml')

In [24]:
print(my_env)



conda_file:
  channels:
  - conda-forge
  dependencies:
  - python=3.8
  - pandas
  - pip
  - pip:
    - azureml-mlflow
  name: my_custom_env
description: Environment created from a Docker image plus Conda environment.
image: mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
name: docker-image-plus-conda-example
tags: {}



In [30]:
ml_client.environments.create_or_update(my_env)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/1b78932d-85c1-434a-abec-b9aa3cef3c79/resourceGroups/azuregigaml/providers/Microsoft.MachineLearningServices/workspaces/azuregigaml-ws/environments/docker-image-plus-conda-example/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f634726f7f0>, 'serialize': <msrest.serialization.Serializer object at 0x7f63470f9090>, 'version': '1', 'latest_version': None, 'conda_file': OrderedDict([('channels', ['defaults', 'conda-forge']), ('dependencies', ['python=3.8', 'pandas', 'pip', OrderedDict([('pip', ['azureml-mlflow'])])]), ('name', 'my_custom_env')]), 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'build': None, 'inference_config': None, 'os_type': 'Linux'

In [26]:
from azure.ai.ml import load_job

simple_mlflow_job = load_job('../definitions/jobs/simple_python_job/definition.yml')

In [27]:
# CLI equivalent is
#   az ml job create -f <definition file>

ml_client.jobs.create_or_update(simple_mlflow_job)

Uploading src (0.0 MBs): 100%|██████████| 977/977 [00:00<00:00, 10080.92it/s]




Experiment,Name,Type,Status,Details Page
mlflow-starter,olive_pot_1cjlc7bkyk,command,Starting,Link to Azure Machine Learning studio


In [7]:
# load a pipeline definied in a yaml file
from azure.ai.ml import load_job

simple_pipeline = load_job('../definitions/pipelines/simple_pipeline/definition.yml')
print(simple_pipeline)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


display_name: hello_pipeline_io
type: pipeline
jobs:
  hello_job:
    type: command
    component:
      name: azureml_anonymous
      version: '1'
      type: command
      outputs:
        world_output:
          type: uri_folder
      command: echo "hello" && echo "world" > ${{outputs.world_output}}/world.txt
      environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest
      is_deterministic: true
    compute: azureml:cpu-cluster
  world_job:
    type: command
    inputs:
      world_input:
        path: ${{parent.jobs.hello_job.outputs.world_output}}
    component:
      name: azureml_anonymous
      version: '1'
      type: command
      inputs:
        world_input:
          type: uri_folder
      command: cat ${{inputs.world_input}}/world.txt
      environment: azureml:docker-image-plus-conda-example:1
      is_deterministic: true
    compute: azureml:cpu-cluster



In [8]:
ml_client.jobs.create_or_update(simple_pipeline)